<a href="https://colab.research.google.com/github/FujunhaoFc/Word2Vec/blob/main/Word2vecPart3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 3 More Fun with Word Vectors

In [1]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

In [6]:
type(model.wv.vectors)

numpy.ndarray

In [7]:
model.wv.vectors.shape

(8630, 300)

In [35]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    #
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index_to_key)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model.wv[word])
    #
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate
    # the average feature vector for each one and return a 2D numpy array

    #
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    #
    # Loop through the reviews
    for i, review in enumerate(reviews):
       #
       # Print status message
       if i % 1000 == 0:
           print("Review %d of %d" % (i, len(reviews)))
       #
       # Use the integer index 'i'
       reviewFeatureVecs[i] = makeFeatureVec(review, model, num_features)

    return reviewFeatureVecs

In [36]:
train = pd.read_csv( "word2vec/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "word2vec/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "word2vec/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

In [37]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [38]:

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [39]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality
min_word_count = 40   # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

In [40]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 12389
Review 1000 of 12389
Review 2000 of 12389
Review 3000 of 12389
Review 4000 of 12389
Review 5000 of 12389
Review 6000 of 12389
Review 7000 of 12389
Review 8000 of 12389
Review 9000 of 12389
Review 10000 of 12389
Review 11000 of 12389
Review 12000 of 12389
Creating average feature vecs for test reviews
Review 0 of 14451
Review 1000 of 14451
Review 2000 of 14451
Review 3000 of 14451
Review 4000 of 14451
Review 5000 of 14451
Review 6000 of 14451
Review 7000 of 14451
Review 8000 of 14451
Review 9000 of 14451
Review 10000 of 14451
Review 11000 of 14451
Review 12000 of 14451
Review 13000 of 14451
Review 14000 of 14451


In [42]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print ("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results
result = forest.predict( testDataVecs )

# Write the test results
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [46]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.vectors
num_clusters = int(word_vectors.shape[0] / 5)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  29.358959436416626 seconds.


In [47]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number
word_centroid_map = dict(zip( model.wv.index_to_key, idx ))

In [49]:
for cluster in range(0,10):
    #
    # Print the cluster number
    print ("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print (words)


Cluster 0
['aiming', 'aims']

Cluster 1
['lou', 'jenny', 'sharon', 'richards', 'burke', 'ma', 'kathleen', 'gina', 'bennett', 'drake', 'angie', 'peters', 'carlos', 'stacy', 'scarlet', 'shannon', 'dunn', 'piper', 'josie', 'shrink', 'tina', 'kristofferson', 'connie']

Cluster 2
['yawn', 'shoe', 'sweat', 'mobile', 'jigsaw']

Cluster 3
['london', 'paris', 'mexico', 'california', 'san', 'los', 'university', 'angeles', 'chicago', 'francisco', 'vegas', 'brooklyn', 'florida', 'berlin', 'nyc', 'records', 'northern', 'rome', 'ny', 'boston', 'kansas', 'arizona', 'madrid', 'port']

Cluster 4
['done', 'developed', 'executed', 'handled', 'constructed']

Cluster 5
['emotional', 'depth', 'emotion', 'genuine', 'credibility', 'intensity', 'dignity', 'warmth', 'complexity', 'pathos', 'sincerity', 'sensitivity', 'seriousness', 'nuances', 'maturity', 'flavor', 'vulnerability', 'sophistication', 'sweetness']

Cluster 6
['inducing']

Cluster 7
['ominous', 'electronic']

Cluster 8
['help', 'hurt', 'allow', 's

In [50]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [53]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [54]:
# Fit a random forest and extract predictions
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print ("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
